<a href="https://colab.research.google.com/github/SamanMansoor/Data-Science/blob/main/Convert_VGG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task Convert into Yolo Format**

In [22]:
import json

# Set the class mapping dictionary
class_mapping = {"dog": 0, "cat": 1}

# Set the output file paths
json_file_path = "/content/cat_dog_annotations.json"
output_file_path = "output.txt"

# Load the JSON file
with open(json_file_path, "r") as f:
    data = json.load(f)

# Loop over the images in the JSON
for image_id, (image_filename, image_data) in enumerate(data["_via_img_metadata"].items()):
    # Get the image width and height
    #default_height=[]
    #default_width=[]
    width = image_data.get("width", 300)
    height = image_data.get("height", 600)

    # Open the output file for this image
    output_filename = f"{image_id}.txt"
    with open(output_filename, "w") as output_file:
        # Loop over the regions in the image
        for region in image_data["regions"]:
            # Get the class name and convert it to an integer
            class_name = list(region["region_attributes"]["class"].keys())[0]
            class_id = class_mapping[class_name]

            # Get the bounding box coordinates and convert them to YOLO format
            x = region["shape_attributes"]["x"]
            y = region["shape_attributes"]["y"]
            w = region["shape_attributes"]["width"]
            h = region["shape_attributes"]["height"]
            x_center = x + w / 2
            y_center = y + h / 2
            x_center /= width
            y_center /= height
            w /= width
            h /= height

            # Write the object detection annotation to the output file
            output_file.write(f"{class_id} {x_center} {y_center} {w} {h}\n")


#**Task2- Convert VGG Via Annotation file into COCO Format**

In [25]:
import json
from pycocotools import mask as maskUtils
from PIL import Image

# Load the VIA annotations from the JSON file
with open('/content/cat_dog_annotations.json') as f:
    data = json.load(f)

# Initialize the COCO dataset object
dataset = {
    "info": {},
    "licenses": [],
    "categories": [{"id": 1, "name": "dog"}, {"id": 2, "name": "cat"}],
    "images": [],
    "annotations": []
}

# Add images and annotations to the dataset object
image_id = 1
annotation_id = 1
for filename in data['_via_img_metadata']:
    # Add the image to the dataset
    img = Image.open("/content/1.jpeg")
    width, height = img.size
    dataset['images'].append({
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": filename
    })

    # Add annotations to the dataset
    for region in data['_via_img_metadata'][filename]['regions']:
        x, y, w, h = region['shape_attributes']['x'], region['shape_attributes']['y'], region['shape_attributes']['width'], region['shape_attributes']['height']
        category = region['region_attributes']['class']
        if category == {'dog': True}:
            category_id = 1
        elif category == {'cat': True}:
            category_id = 2
        else:
            continue
        area = w * h
        bbox = [x, y, w, h]
        segmentation = [[x, y, x+w, y, x+w, y+h, x, y+h]]
        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "bbox": bbox,
            "area": area,
            "segmentation": segmentation,
            "iscrowd": 0
        }
        dataset['annotations'].append(annotation)
        annotation_id += 1

    image_id += 1

# Save the dataset object to a JSON file
with open('coco_annotations.json', 'w') as f:
    json.dump(dataset, f)
